In [6]:
!pip install facenet-pytorch
!pip install torch torchvision
!pip install mtcnn

  Using cached facenet_pytorch-2.6.0-py3-none-any.whl.metadata (12 kB)
  Using cached torch-2.2.2-cp311-cp311-manylinux1_x86_64.whl.metadata (25 kB)
  Using cached torchvision-0.17.2-cp311-cp311-manylinux1_x86_64.whl.metadata (6.6 kB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_nccl_cu12-2.19.3-py3-none-manylinux1_x86_64.whl.metadata (1.8 kB)
  Using cached triton-2.2.0-cp311-cp311-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (1.4 kB)
Using cached facenet_pytorch-2.6.0-py3-none-any.whl (1.9 MB)
   ╸━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.7/755.6 MB 401.6 kB/s eta 0:30:55^C
   ╸━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.7/755.6 MB 401.6 kB/s eta 0:30:55
ERROR: Operation cancelled by user


In [7]:
import torch
from facenet_pytorch import InceptionResnetV1, MTCNN
from PIL import Image, UnidentifiedImageError
import cv2

# Load the pre-trained FaceNet model
model = InceptionResnetV1(pretrained='vggface2').eval()

# Initialize MTCNN for face detection
mtcnn = MTCNN(image_size=160, margin=0)

ModuleNotFoundError: No module named 'facenet_pytorch'

In [ ]:
def preprocess_image(image_path):
    image = cv2.imread(image_path)
    face = mtcnn(image)
    if face is None:
        raise ValueError("No face detected in the image.")
    return face

In [8]:
import os
database_dir = '../lfw_funneled'

# os.listdir(database_dir)

In [ ]:
database = {}

for person_name in os.listdir(database_dir):
    person_dir = os.path.join(database_dir, person_name)
    if os.path.isdir(person_dir):
        embeddings = []
        for file_name in os.listdir(person_dir):
            if file_name.endswith(('jpg', 'jpeg', 'png')):
                image_path = os.path.join(person_dir, file_name)
                try:
                    face = preprocess_image(image_path)
                    face = face.unsqueeze(0)  # Add batch dimension
                    embedding = model(face).detach()  # Generate embedding
                    embeddings.append(embedding)
                except ValueError as e:
                    print(f"Skipping {file_name} in {person_name}: {e}")
        if embeddings:
            database[person_name] = torch.stack(embeddings).mean(dim=0)

print(f"Database size: {len(database)}")


In [ ]:
def euclidean_distance(embedding1, embedding2):
    return torch.dist(embedding1, embedding2).item()

def find_closest_match(embedding, database, threshold=1.1):
    closest_distance = float('inf')
    closest_name = None
    for name, db_embedding in database.items():
        distance = euclidean_distance(embedding, db_embedding)
        if distance < closest_distance:
            closest_distance = distance
            closest_name = name
    if closest_distance < threshold:
        return closest_name, closest_distance
    else:
        return None, closest_distance

In [3]:
# Path to the new image to be matched
new_image_path = '/kaggle/input/test-set/adam_rich.jpeg'

# Preprocess the new image
new_face = preprocess_image(new_image_path)
new_face = new_face.unsqueeze(0)  # Add batch dimension
new_embedding = model(new_face).detach()  # Generate embedding

# Find the closest match in the database
name, distance = find_closest_match(new_embedding, database)
if name:
    print(f"Match found: {name} with distance {distance}")
else:
    print("No match found within the threshold.")

NameError: name 'preprocess_image' is not defined